In [1]:
import pandas as pd
import numpy as np

In [2]:
FIFA = pd.read_csv('fifa21_train.csv')

In [3]:
FIFA.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [4]:
def cleanHightWeightFoot(df):
    #Height
    player_info_part_clean = df.copy()
    def feet2cm (x):
        height = x.split("'")
        return float(height[0])*30.48 + float(height[1][:-1])*2.54  
    player_info_part_clean['height'] = player_info_part_clean['height'].map(feet2cm)

    #Weight
    player_info_part_clean2 = player_info_part_clean.copy()
    def libs2kg (x):
        return float(x[:-3]) * 0.45359237
    player_info_part_clean2['weight'] = player_info_part_clean2['weight'].map(libs2kg)

    # Left and right foot
    player_info_part_clean3 = player_info_part_clean2.copy()
    mapping = {'Right':0,'Left':1}
    player_info_part_clean3 = player_info_part_clean3.replace({'foot': mapping})
    player_info_part_clean3.head()
    
    return player_info_part_clean3

In [5]:
def clean_money(df):
    
    data = df.copy()
    
    def typeConvert(x):
        x = x[1:]
        if 'K' in x:
            x = 1000*float(x[:-1])
        elif "M" in x:
            x = 1000000*float(x[:-1])
        else:
            float(x)
        return str(x)

    data['release clause'] = data['release clause'].map(typeConvert)
    data['release clause'] = data['release clause'].astype(float)

    data['wage'] = data['wage'].map(typeConvert)
    data['wage'] = data['wage'].astype(float)

    data['value'] = data['value'].map(typeConvert)
    data['value'] = data['value'].astype(float)
    
    return data

In [6]:
#This cleans the player_info 
def clean_player(x):
    x = x.drop(['Loan Date End'],axis=1)
    cols = []
    for i in range(len(x.columns)):
        cols.append(x.columns[i].lower())
    x.columns = cols
        
    x.position.fillna(x.bp, inplace=True)
    x.club.fillna(x['team & contract'], inplace=True)
    x = x.drop(['joined','team & contract'],axis=1)
    
    x = cleanHightWeightFoot(x)
    x = clean_money(x)
    
    return x

In [7]:
clean_player(FIFA)

,id,name,age,nationality,club,bp,position,height,weight,foot,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,185.42,76.203518,1,...,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,185.42,78.925072,0,...,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,167.64,64.863709,0,...,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,CM,187.96,81.193034,0,...,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68


In [8]:
#This cleans the skills part of df

def clean_skills(df):
    skills = df.copy()
    #Dropping columns
    skills = skills.drop(['attacking', 'skill', 'movement', 'power', 'mentality', 'defending', 'goalkeeping'], axis = 1)

    #Dropping NaNs
    skills = skills.dropna()
    skills = skills.reset_index(drop = True)

    #Replacing NaNs for the mean
    skills = skills.fillna(skills.mean())
    skills['composure'] = skills['composure'].round(decimals = 1)
    
    return skills

In [9]:
def cleanPlayerBaseStats(df):
    # Meaning of the columns:
    # W/F - Weak Foot
    # SM - Skill Moves
    # A/W - Attacking Work Rate
    # D/W - Defending Work Rate
    # IR -  Interception rate?
    # PAC - Pace .. player speed
    # SHO - Shoot
    # DRI - Dribling
    # DEF - Defense
    # PHY - Physical
    # Hits - 
    player_base_stats = df.copy()
   
    player_base_stats = player_base_stats.dropna()
    player_base_stats.reset_index(drop = True)

    player_base_stats_clean = player_base_stats.copy()
    removeStars1 = lambda x: x[:-1]
    removeStars2 = lambda x: x[:-2]
    player_base_stats_clean['w/f'] = player_base_stats_clean['w/f'].map(removeStars2)
    player_base_stats_clean['sm'] = player_base_stats_clean['sm'].map(removeStars1)
    player_base_stats_clean['ir'] = player_base_stats_clean['ir'].map(removeStars2)

    # We change names by data for some features (the ones that can be ordered)
    player_base_stats_clean2 = player_base_stats_clean.copy()
    mapping = {'Low':0,'Medium':1,'High':2}
    player_base_stats_clean2 = player_base_stats_clean2.replace({'a/w': mapping})
    player_base_stats_clean2 = player_base_stats_clean2.replace({'d/w': mapping})

    # We convert them to numeric values
    player_base_stats_clean3= player_base_stats_clean2.copy()
    player_base_stats_clean3['w/f'] = player_base_stats_clean3['w/f'].astype(float)
    player_base_stats_clean3['sm'] = player_base_stats_clean3['sm'].astype(float)
    player_base_stats_clean3['ir'] = player_base_stats_clean3['ir'].astype(float)

    def typeConvert(x):
        if 'K' in x:
            x = 1000*float(x[:-1])
        else:
            float(x)
        return str(x) 

    player_base_stats_clean3['hits'] = player_base_stats_clean3['hits'].map(typeConvert)
    player_base_stats_clean3['hits'] = player_base_stats_clean3['hits'].astype(float)

    return player_base_stats_clean3

In [10]:
def clean_positions(df):
    
    df_remaining = df.drop(['ls',  'st',  'rs',  'lw',  'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
              'lm', 'lcm', 'cm', 'rcm',  'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
              'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova'], axis=1)
    
    #Get the relevant data
    df_edit = df[['ls',  'st',  'rs',  'lw',  'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
              'lm', 'lcm', 'cm', 'rcm',  'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
              'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova']]
    
    pos = df_edit.copy()
    #Create splits and add to the df
    pos[['ls','ls_add']] = pos.ls.str.split('+',1).tolist()
    pos[['st','st_add']] = pos.st.str.split('+',1).tolist()
    pos[['rs','rs_add']] = pos.rs.str.split('+',1).tolist()
    pos[['lw','lw_add']] = pos.lw.str.split('+',1).tolist()
    pos[['lf','lf_add']] = pos.lf.str.split('+',1).tolist()
    pos[['cf','cf_add']] = pos.cf.str.split('+',1).tolist()
    pos[['rf','rf_add']] = pos.rf.str.split('+',1).tolist()
    pos[['rw','rw_add']] = pos.rw.str.split('+',1).tolist()
    pos[['lam','lam_add']] = pos.lam.str.split('+',1).tolist()
    pos[['cam','cam_add']] = pos.cam.str.split('+',1).tolist()
    pos[['ram','ram_add']] = pos.ram.str.split('+',1).tolist()
    pos[['lm','lm_add']] = pos.lm.str.split('+',1).tolist()
    pos[['lcm','lcm_add']] = pos.lcm.str.split('+',1).tolist()
    pos[['cm','cm_add']] = pos.cm.str.split('+',1).tolist()
    pos[['rcm','rcm_add']] = pos.rcm.str.split('+',1).tolist()
    pos[['rm','rm_add']] = pos.rm.str.split('+',1).tolist()
    pos[['lwb','lwb_add']] = pos.lwb.str.split('+',1).tolist()
    pos[['ldm','ldm_add']] = pos.ldm.str.split('+',1).tolist()
    pos[['cdm','cdm_add']] = pos.cdm.str.split('+',1).tolist()
    pos[['rwb','rwb_add']] = pos.rwb.str.split('+',1).tolist()
    pos[['lb','lb_add']] = pos.lb.str.split('+',1).tolist()
    pos[['lcb','lcb_add']] = pos.lcb.str.split('+',1).tolist()
    pos[['rcb','rcb_add']] = pos.rcb.str.split('+',1).tolist()
    pos[['rb','rb_add']] = pos.rb.str.split('+',1).tolist()
    pos[['gk','gk_add']] = pos.gk.str.split('+',1).tolist()
    
    
    #Convert all columns to numeric
    for a in pos:
        pos[a] =  pd.to_numeric(pos[a], errors='coerce')
    
    #Select only columns that are relevant in respect to multicollinearity
    pos_spl_final = pos[['st',  'lw', 'cf', 'cam', 'lm','cm', 'lwb', 'cdm',  'lb', 'lcb', 'gk', 'st_add',
                             'lw_add', 'cf_add','cam_add', 'lm_add', 'cm_add', 'lwb_add', 'cdm_add', 'lb_add', 
                             'lcb_add','gk_add','ova']]
    
    df_new = pd.concat([df_remaining,pos_spl_final], axis=1)    
    
    return df_new

# Complete Function for Cleaning

In [11]:

#Global Function 
def preprocess(df):
    df_clean = df.copy()
    df_clean = clean_player(df_clean)
    df_clean = clean_skills(df_clean)
    df_clean = cleanPlayerBaseStats(df_clean)
    df_clean = clean_positions(df_clean)
    
    return df_clean

In [12]:
print(FIFA.shape)
preprocess(FIFA)

(11701, 101)


/var/folders/gg/ydzntksx7jl3t92y3_bqsw140000gn/T/ipykernel_14438/1956186835.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skills = skills.fillna(skills.mean())


,id,name,age,nationality,club,bp,position,height,weight,foot,...,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,...,0,1,1,1,1,1,1,1,1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,...,0,1,1,2,2,2,2,2,2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,...,0,0,1,2,2,2,2,2,2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,...,0,2,2,2,2,2,2,2,2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,...,0,2,2,2,2,2,2,2,2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11417,196848,I. Ávila,29,Peru,Melgar FBC,ST,ST,170.18,69.853225,0,...,0,2,2,2,2,2,2,2,2,65
11418,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,185.42,76.203518,1,...,0,2,2,2,2,2,2,2,2,65
11419,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,185.42,78.925072,0,...,0,2,2,2,2,2,2,2,2,70
11420,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,167.64,64.863709,0,...,0,2,2,2,2,2,2,2,2,67


In [13]:
FIFA_preprocessed = preprocess(FIFA)

/var/folders/gg/ydzntksx7jl3t92y3_bqsw140000gn/T/ipykernel_14438/1956186835.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skills = skills.fillna(skills.mean())


In [14]:
pd.set_option("display.max_columns", None)
FIFA_preprocessed.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,value,wage,release clause,contract,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,175.26,73.028372,0,1,525000.0,4000.0,801000.0,2015 ~ 2020,54,47,43,70,44.0,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,7,12,14,9,6,1682,357,4.0,2.0,2,1,1.0,69,51,63,63,51,60,3.0,58,61,62,63,63,63,59,59,58,54,15,1,0,0,1,1,1,1,1,1,1,1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,182.88,72.121187,0,0,8500000.0,23000.0,0.0,"Dec 31, 2020 On Loan",66,79,76,68,76.0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,11,7,14,7,16,1961,412,3.0,4.0,2,0,2.0,83,75,68,82,33,71,44.0,77,77,77,76,76,68,57,53,53,48,18,0,0,0,1,1,2,2,2,2,2,2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,162.56,60.781378,0,0,9000000.0,49000.0,15300000.0,2019 ~ 2022,73,76,34,78,75.0,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,6,3,6,3,3,1925,404,4.0,4.0,2,1,2.0,80,77,78,86,27,56,73.0,73,80,79,80,79,74,59,56,53,41,12,2,0,0,0,1,2,2,2,2,2,2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,177.80,68.946040,0,13,275000.0,4000.0,694000.0,2016 ~ 2021,44,42,58,62,36.0,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,8,9,6,7,12,1527,329,2.0,2.0,1,1,1.0,57,44,54,57,57,60,7.0,50,51,51,53,53,56,56,58,57,58,14,2,0,0,2,2,2,2,2,2,2,2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,180.34,68.038855,0,8,725000.0,2000.0,1400000.0,2018 ~ 2021,49,37,61,68,34.0,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,8,9,15,5,15,1664,360,2.0,3.0,0,1,1.0,66,44,60,64,60,66,4.0,56,59,59,61,62,63,64,64,63,61,15,2,0,0,2,2,2,2,2,2,2,2,65


# Start of Day 2

In [15]:
#First step is X - y split
#dropped some extra columns because they werent necesssary for analysiis
X = FIFA_preprocessed.drop(['id','name', 'contract', 'position','ova','nationality','club'], axis = 1)
y= FIFA_preprocessed[['ova']]

In [16]:
#First step is Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=90)

In [17]:
X_train

,age,bp,height,weight,foot,growth,value,wage,release clause,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add
1980,26,CAM,177.80,74.842741,0,1,0.0,0.0,0.0,59,48,48,67,55.0,68,55.0,42,65,67,69,68,78.0,68,79.0,65,52.0,67,38,63,49,50.0,65.0,67.0,68,62.0,57,45,35.0,8,8,11,11,6,1701,354,3.0,3.0,1,1,1.0,68,57,63,69,49,48,4.0,60,65,64,66,65,64,58,57,56,50,15,1,0,0,0,1,1,1,1,1,1,1
5170,30,RM,170.18,59.874193,0,0,6500000.0,25000.0,13700000.0,77,72,59,76,64.0,77,76.0,63,74,75,67,66,75.0,74,83.0,68,57.0,67,63,69,57,56.0,75.0,77.0,62,70.0,52,56,53.0,14,15,8,16,11,1954,405,3.0,3.0,1,1,1.0,66,70,75,76,55,63,9.0,71,74,73,74,74,73,66,65,64,59,20,2,0,0,1,1,2,2,2,2,2,2
7621,24,ST,177.80,74.842741,0,6,210000.0,3000.0,378000.0,53,52,62,54,56.0,56,52.0,42,50,57,73,75,71.0,63,65.0,64,63.0,66,72,56,59,46.0,58.0,53.0,52,57.0,47,56,55.0,13,7,7,8,14,1677,360,3.0,2.0,1,1,1.0,74,56,52,59,52,67,3.0,60,59,59,57,58,55,57,55,57,57,16,2,0,0,2,2,2,2,2,2,2,2
7239,24,ST,180.34,72.121187,0,5,500000.0,2000.0,763000.0,35,67,46,58,69.0,67,56.0,42,36,66,76,68,83.0,57,79.0,57,65.0,57,58,50,36,21.0,66.0,39.0,62,57.0,29,16,19.0,15,9,12,7,7,1530,324,3.0,3.0,1,0,1.0,72,61,45,68,24,54,5.0,62,61,62,59,59,51,43,40,41,36,15,1,0,0,1,1,1,1,1,1,1,1
3411,30,CB,180.34,86.182550,0,0,3800000.0,50000.0,0.0,64,47,64,76,68.0,70,44.0,42,66,72,66,69,72.0,70,71.0,74,75.0,78,86,63,92,77.0,53.0,62.0,52,77.0,74,78,78.0,6,6,12,9,9,1945,422,3.0,2.0,1,2,2.0,68,57,66,71,75,85,4.0,64,65,65,66,67,69,73,75,73,77,15,1,0,0,1,1,1,1,1,1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9539,32,CB,190.50,78.925072,1,0,300000.0,2000.0,525000.0,30,27,64,55,16.0,34,19.0,29,48,51,51,54,55.0,51,42.0,42,73.0,47,74,17,71,63.0,23.0,37.0,37,57.0,64,64,63.0,10,12,13,13,8,1357,296,2.0,2.0,1,2,1.0,53,28,42,42,64,67,5.0,42,40,40,42,42,46,53,57,55,64,15,2,0,0,2,2,2,2,2,2,0,2
6815,25,CAM,175.26,73.028372,0,3,8500000.0,8000.0,13000000.0,75,72,39,76,75.0,73,73.0,72,73,77,73,70,84.0,72,75.0,79,55.0,73,57,77,59,38.0,74.0,75.0,69,74.0,51,49,38.0,10,7,16,13,7,1926,402,4.0,3.0,2,1,1.0,71,75,75,75,45,61,30.0,70,74,74,75,74,72,62,61,59,51,16,2,0,0,2,2,2,2,2,2,2,2
5746,20,CAM,170.18,64.863709,0,9,120000.0,550.0,216000.0,48,42,48,63,40.0,60,48.0,39,67,61,60,69,63.0,56,83.0,52,50.0,40,40,37,44,24.0,49.0,55.0,44,57.0,37,33,43.0,6,12,5,10,15,1443,303,3.0,2.0,1,1,1.0,65,43,57,62,35,41,4.0,52,56,55,57,57,54,47,47,46,41,15,2,0,0,2,2,2,2,2,2,2,2
2717,29,GK,187.96,84.821773,0,1,25000000.0,67000.0,44400000.0,15,8,17,37,13.0,12,13.0,12,24,23,44,50,49.0,82,52.0,54,74.0,36,75,14,35,18.0,10.0,50.0,20,58.0,11,13,16.0,85,82,72,82,88,1286,456,2.0,1.0,1,1,3.0,85,82,72,88,47,82,83.0,30,28,30,32,30,32,28,30,28,29,83,3,0,0,3,3,3,3,3,3,3,2


In [18]:
numerical_train = X_train.select_dtypes(include=[np.number])
numerical_test = X_test.select_dtypes(include=[np.number])

In [19]:
categorical_train = X_train.select_dtypes(include = [object])
categorical_test = X_test.select_dtypes(include = [object])

#####      · Encoding Categorical Data.  (Not relevant)
def categoricalEncoder(df):
    from sklearn.preprocessing import OneHotEncoder
    cat = df.copy()

    # We define the function that we will use in OneHotColum
    def oneHotColumn(col_transform):
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit()
        onehotlabels = enc.transform(col_transform).toarray()
        return pd.DataFrame(onehotlabels,columns = enc.categories_)

    # We apply OneHotColum to the other features of the frame
    concat_categorical = pd.DataFrame()# categorical[['Customer']]
    for col in cat.columns:
          df_from_column = oneHotColumn(cat[[col]])
          concat_categorical = pd.concat([concat_categorical,df_from_column], axis=1)#
        
    new_columns = []
    for column in concat_categorical.columns:
        new_columns.append(column[0])
    concat_categorical.columns = new_columns
    
    return concat_categorical

In [20]:
categorical_train.columns

Index(['bp'], dtype='object')

In [24]:
# Creating Dummy variables:


from sklearn.preprocessing import OneHotEncoder
encoder_bp = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['bp']])#fit determines how many unique alues has each categorical column. You can see below.
print(encoder_bp.categories_)
encoded_train_bp = encoder_bp.transform(categorical_train[['bp']]).toarray()
encoded_test_bp = encoder_bp.transform(categorical_test[['bp']]).toarray()
bp_encoded_train = pd.DataFrame(encoded_train_bp,columns = encoder_bp.categories_)
bp_encoded_test = pd.DataFrame(encoded_test_bp,columns = encoder_bp.categories_)

categorical_encoded_train = pd.DataFrame(encoded_train_bp,columns = encoder_bp.categories_)
categorical_encoded_test = pd.DataFrame(encoded_test_bp,columns = encoder_bp.categories_)

#categorical_encoded_train = pd.concat([nationality_encoded_train, club_encoded_train, bp_encoded_train], axis = 1)
#categorical_encoded_test = pd.concat([nationality_encoded_test, club_encoded_test, bp_encoded_test], axis = 1)




[array(['CAM', 'CB', 'CDM', 'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB',
       'RB', 'RM', 'RW', 'RWB', 'ST'], dtype=object)]


In [26]:
categorical_encoded_train

,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7990,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7991,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7992,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7993,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
print(categorical_train.shape)
print(categorical_test.shape)
print(categorical_encoded_train.shape)
print(categorical_encoded_test.shape)

(7995, 1)
(3427, 1)
(7995, 15)
(3427, 15)


In [28]:
#Now is the normalization of numerical data with MinMaxScaler()
#fit train set
#transform both train and test set
from sklearn.preprocessing import MinMaxScaler 
transformer = MinMaxScaler().fit(numerical_train)
x_normalized_train = transformer.transform(numerical_train)
x_normalized_test = transformer.transform(numerical_test)
x_normalized_train = pd.DataFrame(x_normalized_train, columns=numerical_train.columns)
x_normalized_test = pd.DataFrame(x_normalized_test, columns=numerical_train.columns)


In [29]:
x_normalized_test.head()

,age,height,weight,foot,growth,value,wage,release clause,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add
0,0.111111,0.65,0.537815,1.0,0.500000,0.009167,0.001339,0.009009,0.556818,0.641304,0.534091,0.686747,0.465116,0.560440,0.477778,0.404494,0.666667,0.617978,0.590361,0.607143,0.378049,0.492308,0.455696,0.638554,0.400000,0.741176,0.684211,0.622222,0.448276,0.529412,0.677419,0.666667,0.535714,0.642857,0.471910,0.476190,0.481928,0.045455,0.081395,0.139535,0.078652,0.045455,0.568976,0.488550,0.25,0.25,0.5,0.5,0.0,0.521739,0.561644,0.544118,0.432836,0.460526,0.62500,0.000000,0.613333,0.602564,0.597403,0.610390,0.613333,0.638889,0.608696,0.600000,0.608696,0.513889,0.063291,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
1,0.296296,0.50,0.554622,0.0,0.192308,0.041111,0.014286,0.046246,0.704545,0.369565,0.647727,0.650602,0.395349,0.659341,0.588889,0.382022,0.559524,0.640449,0.939759,0.916667,0.768293,0.523077,0.670886,0.650602,0.614286,0.800000,0.631579,0.600000,0.655172,0.729412,0.548387,0.476190,0.535714,0.607143,0.685393,0.714286,0.734940,0.045455,0.093023,0.069767,0.134831,0.068182,0.689129,0.656489,0.50,0.50,1.0,0.5,0.0,0.913043,0.397260,0.514706,0.567164,0.697368,0.68750,0.004132,0.586667,0.641026,0.584416,0.584416,0.653333,0.611111,0.797101,0.700000,0.811594,0.694444,0.075949,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
2,0.111111,0.50,0.386555,0.0,0.384615,0.001000,0.001786,0.001135,0.397727,0.565217,0.511364,0.397590,0.500000,0.428571,0.422222,0.348315,0.404762,0.438202,0.662651,0.583333,0.536585,0.384615,0.594937,0.530120,0.528571,0.400000,0.447368,0.533333,0.298851,0.176471,0.559140,0.380952,0.595238,0.464286,0.191011,0.178571,0.108434,0.079545,0.139535,0.116279,0.056180,0.136364,0.390973,0.202290,0.50,0.25,0.5,0.5,0.0,0.550725,0.465753,0.264706,0.298507,0.144737,0.34375,0.000000,0.493333,0.461538,0.454545,0.428571,0.440000,0.375000,0.318841,0.271429,0.318841,0.236111,0.088608,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
3,0.037037,0.55,0.310924,0.0,0.538462,0.001667,0.000893,0.002138,0.295455,0.250000,0.556818,0.481928,0.279070,0.406593,0.222222,0.258427,0.452381,0.404494,0.578313,0.619048,0.548780,0.261538,0.544304,0.349398,0.614286,0.611765,0.578947,0.366667,0.574713,0.576471,0.408602,0.345238,0.333333,0.428571,0.561798,0.559524,0.554217,0.090909,0.034884,0.116279,0.056180,0.068182,0.405594,0.267176,0.25,0.25,0.5,1.0,0.0,0.521739,0.178082,0.250000,0.253731,0.539474,0.56250,0.000109,0.360000,0.371795,0.350649,0.363636,0.386667,0.402778,0.521739,0.514286,0.550725,0.541667,0.037975,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
4,0.407407,0.40,0.369748,0.0,0.038462,0.004722,0.003571,0.004276,0.500000,0.576087,0.500000,0.710843,0.476744,0.593407,0.500000,0.528090,0.702381,0.640449,0.710843,0.702381,0.646341,0.415385,0.607595,0.566265,0.571429,0.505882,0.565789,0.588889,0.505747,0.705882,0.580645,0.619048,0.559524,0.440476,0.719101,0.690476,0.674699,0.090909,0.127907,0.127907,0.112360,0.079545,0.638271,0.557252,0.50,0.25,0.5,0.5,0.0,0.652174,0.493151,0.544118,0.507463,0.684211,0.50000,0.000217,0.573333,0.602564,0.584416,0.610390,0.613333,0.638889,0.681159,0.685714,0.695652,0.638889,0.088608,0.833333,0.75,0.714286,0.750,0.875,0.666667,0.833333,0.500000,0.833333,0.857143,0.5


In [30]:
X_transformed_train = pd.concat([x_normalized_train, categorical_encoded_train], axis = 1)
X_transformed_test = pd.concat([x_normalized_test, categorical_encoded_test], axis = 1)

In [31]:
# 5. Modeling
#       · Apply model.
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_transformed_train, y_train)

from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train,y_train)

In [33]:
#Now performing error metrics on train data
#R2 score
from sklearn.metrics import r2_score
y_pred_train = reg.predict(X_transformed_train)
print(r2_score(y_train, y_pred_train))

#MAE ON train data
from sklearn.metrics import mean_absolute_error
mae_train = mean_absolute_error(y_train, y_pred_train)
print(mae_train)

#MSE on train DATA
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, y_pred_train)
print(mse)

#RMSE ON train DATA
import math
rmse = math.sqrt(mse)
print(rmse)

0.9267062827053378
1.4527800578486554
3.4838808725743537
1.8665157038113431


# Formular for Transformation

# Validation

In [34]:
fifa_valid = pd.read_csv('fifa21_validate.csv')

In [35]:
fifa_valid.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",€975K,€5K,€0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,Right,5,"Jan 20, 2020",NaN,€1.2M,€3K,€2.2M,2020 ~ 2020,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,Right,17,"Nov 28, 2018",NaN,€120K,€500,€249K,2018 ~ 2020,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,Right,23,"Jan 10, 2020",NaN,€160K,€500,€464K,2020 ~ 2022,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,Right,5,"Jul 1, 2019",NaN,€2.3M,€13K,€4.3M,2019 ~ 2023,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


In [36]:
fifa_valid_new = preprocess(fifa_valid)

/var/folders/gg/ydzntksx7jl3t92y3_bqsw140000gn/T/ipykernel_14438/1956186835.py:13: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  skills = skills.fillna(skills.mean())


In [37]:
fifa_valid_new.head()

,id,name,age,nationality,club,bp,position,height,weight,foot,growth,value,wage,release clause,contract,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,ova
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,187.96,87.996920,0,7,975000.0,5000.0,0.0,"Jun 30, 2021 On Loan",47,21,62,60,40.0,44,43.0,36,51,54,60,68,63.0,63,49.0,48,77.0,51,87,25,68,62.0,38.0,39.0,39,49.0,68,67,65.0,11,8,15,13,9,1551,334,2.0,2.0,0,2,1.0,64,30,50,50,66,74,34.0,48,48,47,48,50,51,59,61,61,67,16,2,0,0,2,2,2,2,2,2,2,2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,172.72,69.853225,0,5,1200000.0,3000.0,2200000.0,2020 ~ 2020,62,60,44,62,70.0,76,68.0,56,60,70,77,72,83.0,64,79.0,76,62.0,63,47,75,72,26.0,64.0,64.0,60,65.0,19,23,19.0,14,13,9,9,8,1726,358,4.0,3.0,2,0,1.0,74,67,62,74,24,57,12.0,64,68,68,68,67,61,49,47,45,38,17,2,0,0,2,2,2,2,2,2,2,2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,187.96,79.832257,0,17,120000.0,500.0,249000.0,2018 ~ 2020,7,5,11,21,4.0,6,8.0,8,20,10,28,25,33.0,41,38.0,40,49.0,22,54,6,20,9.0,7.0,26.0,14,31.0,8,9,10.0,56,52,53,53,55,808,295,2.0,1.0,1,1,1.0,56,52,53,55,26,53,3.0,18,15,17,17,16,18,16,18,16,18,53,2,0,0,2,2,2,2,2,2,2,2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,190.50,77.110703,0,23,160000.0,500.0,464000.0,2020 ~ 2022,38,31,55,59,32.0,51,34.0,38,47,54,59,58,56.0,48,54.0,48,48.0,60,58,28,61,55.0,33.0,40.0,41,59.0,53,52,54.0,7,5,13,5,6,1381,303,3.0,2.0,1,1,1.0,58,34,47,52,53,59,6.0,46,47,46,47,49,49,53,54,53,54,11,2,0,0,2,2,2,2,2,2,2,2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,187.96,77.110703,0,5,2300000.0,13000.0,4300000.0,2019 ~ 2023,57,59,45,78,56.0,71,57.0,51,74,74,68,66,66.0,64,56.0,73,56.0,74,72,62,66,78.0,53.0,62.0,55,63.0,72,68,71.0,12,7,13,15,13,1864,407,4.0,2.0,1,1,1.0,67,62,68,70,69,71,45.0,63,66,66,68,67,70,70,72,69,68,18,2,0,0,2,2,2,2,2,2,2,2,70


In [38]:
#First step is X - y split
#dropped some extra columns because they werent necesssary for analysiis
X_2 = fifa_valid_new.drop(['id','name', 'contract', 'position','ova'], axis = 1)
y_2 = fifa_valid_new[['ova']]

In [39]:
#First step is Train-test split
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=90)

In [40]:
numerical_new = X_2.select_dtypes(include=[np.number])
categorical_new =X_2.select_dtypes(include = object)

In [41]:
display(numerical_new.head())
display(categorical_new.head())

,age,height,weight,foot,growth,value,wage,release clause,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add
0,23,187.96,87.996920,0,7,975000.0,5000.0,0.0,47,21,62,60,40.0,44,43.0,36,51,54,60,68,63.0,63,49.0,48,77.0,51,87,25,68,62.0,38.0,39.0,39,49.0,68,67,65.0,11,8,15,13,9,1551,334,2.0,2.0,0,2,1.0,64,30,50,50,66,74,34.0,48,48,47,48,50,51,59,61,61,67,16,2,0,0,2,2,2,2,2,2,2,2
1,22,172.72,69.853225,0,5,1200000.0,3000.0,2200000.0,62,60,44,62,70.0,76,68.0,56,60,70,77,72,83.0,64,79.0,76,62.0,63,47,75,72,26.0,64.0,64.0,60,65.0,19,23,19.0,14,13,9,9,8,1726,358,4.0,3.0,2,0,1.0,74,67,62,74,24,57,12.0,64,68,68,68,67,61,49,47,45,38,17,2,0,0,2,2,2,2,2,2,2,2
2,19,187.96,79.832257,0,17,120000.0,500.0,249000.0,7,5,11,21,4.0,6,8.0,8,20,10,28,25,33.0,41,38.0,40,49.0,22,54,6,20,9.0,7.0,26.0,14,31.0,8,9,10.0,56,52,53,53,55,808,295,2.0,1.0,1,1,1.0,56,52,53,55,26,53,3.0,18,15,17,17,16,18,16,18,16,18,53,2,0,0,2,2,2,2,2,2,2,2
3,16,190.50,77.110703,0,23,160000.0,500.0,464000.0,38,31,55,59,32.0,51,34.0,38,47,54,59,58,56.0,48,54.0,48,48.0,60,58,28,61,55.0,33.0,40.0,41,59.0,53,52,54.0,7,5,13,5,6,1381,303,3.0,2.0,1,1,1.0,58,34,47,52,53,59,6.0,46,47,46,47,49,49,53,54,53,54,11,2,0,0,2,2,2,2,2,2,2,2
4,24,187.96,77.110703,0,5,2300000.0,13000.0,4300000.0,57,59,45,78,56.0,71,57.0,51,74,74,68,66,66.0,64,56.0,73,56.0,74,72,62,66,78.0,53.0,62.0,55,63.0,72,68,71.0,12,7,13,15,13,1864,407,4.0,2.0,1,1,1.0,67,62,68,70,69,71,45.0,63,66,66,68,67,70,70,72,69,68,18,2,0,0,2,2,2,2,2,2,2,2


,nationality,club,bp
0,United States,FK Austria Wien,CB
1,Kosovo,Heart of Midlothian,CAM
2,United States,Real Salt Lake,GK
3,England,Leeds United,CDM
4,Switzerland,1. FSV Mainz 05,CDM


In [42]:
#from sklearn.preprocessing import MinMaxScaler 
#transformer = MinMaxScaler().fit()
x_normalized_new = transformer.transform(numerical_new)

In [45]:
#One hot encoding
#from sklearn.preprocessing import OneHotEncoder
#encoder_nationality = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['nationality']])#fit determines how many unique alues has each categorical column. You can see below.
#print(encoder_nationality.categories_)
#encoded_nationality_new = encoder_nationality.transform(categorical_new[['nationality']]).toarray()
#encoded_test_nationality = encoder_nationality.transform(categorical_test[['nationality']]).toarray()
#nationality_encoded_new = pd.DataFrame(encoded_nationality_new,columns = encoder_nationality.categories_)
#nationality_encoded_test = pd.DataFrame(encoded_test_nationality,columns = encoder_nationality.categories_)

#encoder_club = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['club']])#fit determines how many unique alues has each categorical column. You can see below.
#encoded_club_new = encoder_club.transform(categorical_new[['club']]).toarray()
#encoded_test_club = encoder_club.transform(categorical_test[['club']]).toarray()
#club_encoded_new = pd.DataFrame(encoded_club_new,columns = encoder_club.categories_)
#club_encoded_test = pd.DataFrame(encoded_test_club,columns = encoder_club.categories_)

#encoder_bp = OneHotEncoder(handle_unknown = 'ignore').fit(categorical_train[['bp']])#fit determines how many unique alues has each categorical column. You can see below.
#print(encoder_bp.categories_)
encoded_bp_new = encoder_bp.transform(categorical_new[['bp']]).toarray()
#encoded_test_bp = encoder_bp.transform(categorical_test[['bp']]).toarray()
bp_encoded_new = pd.DataFrame(encoded_bp_new,columns = encoder_bp.categories_)
#bp_encoded_test = pd.DataFrame(encoded_test_bp,columns = encoder_bp.categories_)

#categorical_encoded_new = pd.concat([nationality_encoded_new, club_encoded_new, bp_encoded_new], axis = 1)


categorical_encoded_new = pd.DataFrame(encoded_bp_new,columns = encoder_bp.categories_)
categorical_encoded_new.head()

#categorical_encoded_test = pd.concat([nationality_encoded_test, club_encoded_test, bp_encoded_test], axis = 1)

,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
#Now is the normalization of numerical data with MinMaxScaler()
#fit train set
#transform both train and test set
#from sklearn.preprocessing import MinMaxScaler 
#transformer = MinMaxScaler().fit(numerical_train)
x_normalized_new = transformer.transform(numerical_new)
#x_normalized_test = transformer.transform(numerical_test)
x_normalized_new = pd.DataFrame(x_normalized_new, columns=numerical_train.columns)
#x_normalized_test = pd.DataFrame(x_normalized_test, columns=numerical_train.columns)

In [47]:
x_normalized_new.head()

,age,height,weight,foot,growth,value,wage,release clause,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add
0,0.259259,0.65,0.647059,0.0,0.269231,0.010833,0.008929,0.000000,0.465909,0.195652,0.647727,0.590361,0.418605,0.428571,0.433333,0.348315,0.500000,0.528090,0.566265,0.678571,0.597561,0.507692,0.405063,0.433735,0.742857,0.458824,0.881579,0.233333,0.678161,0.670588,0.387097,0.333333,0.369048,0.440476,0.730337,0.726190,0.698795,0.102273,0.069767,0.151163,0.123596,0.079545,0.521297,0.404580,0.25,0.25,0.0,1.0,0.0,0.550725,0.136986,0.367647,0.328358,0.710526,0.734375,0.003371,0.426667,0.435897,0.402597,0.415584,0.453333,0.472222,0.637681,0.642857,0.681159,0.708333,0.088608,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
1,0.222222,0.35,0.310924,0.0,0.192308,0.013333,0.005357,0.013213,0.636364,0.619565,0.443182,0.614458,0.767442,0.780220,0.711111,0.573034,0.607143,0.707865,0.771084,0.726190,0.841463,0.523077,0.784810,0.771084,0.528571,0.600000,0.355263,0.788889,0.724138,0.247059,0.666667,0.630952,0.619048,0.630952,0.179775,0.202381,0.144578,0.136364,0.127907,0.081395,0.078652,0.068182,0.632549,0.496183,0.75,0.50,1.0,0.0,0.0,0.695652,0.643836,0.544118,0.686567,0.157895,0.468750,0.000979,0.640000,0.692308,0.675325,0.675325,0.680000,0.611111,0.492754,0.442857,0.449275,0.305556,0.101266,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
2,0.111111,0.65,0.495798,0.0,0.653846,0.001333,0.000893,0.001495,0.011364,0.021739,0.068182,0.120482,0.000000,0.010989,0.044444,0.033708,0.130952,0.033708,0.180723,0.166667,0.231707,0.169231,0.265823,0.337349,0.342857,0.117647,0.447368,0.022222,0.126437,0.047059,0.053763,0.178571,0.071429,0.226190,0.056180,0.035714,0.036145,0.613636,0.581395,0.593023,0.573034,0.602273,0.048951,0.255725,0.25,0.00,0.5,0.5,0.0,0.434783,0.438356,0.411765,0.402985,0.184211,0.406250,0.000000,0.026667,0.012821,0.012987,0.012987,0.000000,0.013889,0.014493,0.028571,0.028986,0.027778,0.556962,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
3,0.000000,0.70,0.445378,0.0,0.884615,0.001778,0.000893,0.002787,0.363636,0.304348,0.568182,0.578313,0.325581,0.505495,0.333333,0.370787,0.452381,0.528090,0.554217,0.559524,0.512195,0.276923,0.468354,0.433735,0.328571,0.564706,0.500000,0.266667,0.597701,0.588235,0.333333,0.345238,0.392857,0.559524,0.561798,0.547619,0.566265,0.056818,0.034884,0.127907,0.033708,0.045455,0.413223,0.286260,0.50,0.25,0.5,0.5,0.0,0.463768,0.191781,0.323529,0.358209,0.539474,0.500000,0.000326,0.400000,0.423077,0.389610,0.402597,0.440000,0.444444,0.550725,0.542857,0.565217,0.527778,0.025316,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5
4,0.296296,0.65,0.445378,0.0,0.192308,0.025556,0.023214,0.025826,0.579545,0.608696,0.454545,0.807229,0.604651,0.725275,0.588889,0.516854,0.773810,0.752809,0.662651,0.654762,0.634146,0.523077,0.493671,0.734940,0.442857,0.729412,0.684211,0.644444,0.655172,0.858824,0.548387,0.607143,0.559524,0.607143,0.775281,0.738095,0.771084,0.113636,0.058140,0.127907,0.146067,0.125000,0.720280,0.683206,0.75,0.25,0.5,0.5,0.0,0.594203,0.575342,0.632353,0.626866,0.750000,0.687500,0.004567,0.626667,0.666667,0.649351,0.675325,0.680000,0.736111,0.797101,0.800000,0.797101,0.722222,0.113924,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5


In [48]:
#Concatenat

x_transformed_new = pd.concat([x_normalized_new, categorical_encoded_new], axis =1)
x_transformed_new.head()

,age,height,weight,foot,growth,value,wage,release clause,crossing,finishing,heading accuracy,short passing,volleys,dribbling,curve,fk accuracy,long passing,ball control,acceleration,sprint speed,agility,reactions,balance,shot power,jumping,stamina,strength,long shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing tackle,sliding tackle,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,st,lw,cf,cam,lm,cm,lwb,cdm,lb,lcb,gk,st_add,lw_add,cf_add,cam_add,lm_add,cm_add,lwb_add,cdm_add,lb_add,lcb_add,gk_add,"(CAM,)","(CB,)","(CDM,)","(CF,)","(CM,)","(GK,)","(LB,)","(LM,)","(LW,)","(LWB,)","(RB,)","(RM,)","(RW,)","(RWB,)","(ST,)"
0,0.259259,0.65,0.647059,0.0,0.269231,0.010833,0.008929,0.000000,0.465909,0.195652,0.647727,0.590361,0.418605,0.428571,0.433333,0.348315,0.500000,0.528090,0.566265,0.678571,0.597561,0.507692,0.405063,0.433735,0.742857,0.458824,0.881579,0.233333,0.678161,0.670588,0.387097,0.333333,0.369048,0.440476,0.730337,0.726190,0.698795,0.102273,0.069767,0.151163,0.123596,0.079545,0.521297,0.404580,0.25,0.25,0.0,1.0,0.0,0.550725,0.136986,0.367647,0.328358,0.710526,0.734375,0.003371,0.426667,0.435897,0.402597,0.415584,0.453333,0.472222,0.637681,0.642857,0.681159,0.708333,0.088608,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.222222,0.35,0.310924,0.0,0.192308,0.013333,0.005357,0.013213,0.636364,0.619565,0.443182,0.614458,0.767442,0.780220,0.711111,0.573034,0.607143,0.707865,0.771084,0.726190,0.841463,0.523077,0.784810,0.771084,0.528571,0.600000,0.355263,0.788889,0.724138,0.247059,0.666667,0.630952,0.619048,0.630952,0.179775,0.202381,0.144578,0.136364,0.127907,0.081395,0.078652,0.068182,0.632549,0.496183,0.75,0.50,1.0,0.0,0.0,0.695652,0.643836,0.544118,0.686567,0.157895,0.468750,0.000979,0.640000,0.692308,0.675325,0.675325,0.680000,0.611111,0.492754,0.442857,0.449275,0.305556,0.101266,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.111111,0.65,0.495798,0.0,0.653846,0.001333,0.000893,0.001495,0.011364,0.021739,0.068182,0.120482,0.000000,0.010989,0.044444,0.033708,0.130952,0.033708,0.180723,0.166667,0.231707,0.169231,0.265823,0.337349,0.342857,0.117647,0.447368,0.022222,0.126437,0.047059,0.053763,0.178571,0.071429,0.226190,0.056180,0.035714,0.036145,0.613636,0.581395,0.593023,0.573034,0.602273,0.048951,0.255725,0.25,0.00,0.5,0.5,0.0,0.434783,0.438356,0.411765,0.402985,0.184211,0.406250,0.000000,0.026667,0.012821,0.012987,0.012987,0.000000,0.013889,0.014493,0.028571,0.028986,0.027778,0.556962,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.70,0.445378,0.0,0.884615,0.001778,0.000893,0.002787,0.363636,0.304348,0.568182,0.578313,0.325581,0.505495,0.333333,0.370787,0.452381,0.528090,0.554217,0.559524,0.512195,0.276923,0.468354,0.433735,0.328571,0.564706,0.500000,0.266667,0.597701,0.588235,0.333333,0.345238,0.392857,0.559524,0.561798,0.547619,0.566265,0.056818,0.034884,0.127907,0.033708,0.045455,0.413223,0.286260,0.50,0.25,0.5,0.5,0.0,0.463768,0.191781,0.323529,0.358209,0.539474,0.500000,0.000326,0.400000,0.423077,0.389610,0.402597,0.440000,0.444444,0.550725,0.542857,0.565217,0.527778,0.025316,0.833333,0.75,0.714286,0.875,0.875,0.833333,0.833333,0.833333,0.833333,0.857143,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.296296,0.65,0.445378,0.0,0.192308,0.025556,0.023214,0.025826,0.579545,0.608696,0.454545,0.807229,0.604651,0.725275,0.588889,0.516854,0.773810,0.752809,0.662651,0.654762,0.634146,0.523077,0.493671,0.734940,0.442857,0.729412,0.684211,0.644444,0.655172,0.858824,0.548387,0.607143,0.559524,0.607143,0.775281,0.738095,0.771084,0.113636,0.058140,0.127907,0.146067,0.125000,0.720280,0.683206,

In [49]:
reg_predicted = reg.predict(x_transformed_new)

In [50]:
#Now performing error metrics on train data
#R2 score
#from sklearn.metrics import r2_score
reg_predicted = reg.predict(x_transformed_new)
#y_pred_train = reg.predict(X_transformed_train)

print(r2_score(y_2, reg_predicted))
#MAE ON train data
from sklearn.metrics import mean_absolute_error
mae_new = mean_absolute_error(y_2, reg_predicted)
print(mae_new)
#MSE on train DATA
from sklearn.metrics import mean_squared_error
mse_new = mean_squared_error(y_2, reg_predicted)
print(mse_new)
#RMSE ON train DATA
import math
rmse_new = math.sqrt(mse_new)
print(rmse_new)

0.9236041288240088
1.450308175867432
3.45615843839068
1.8590746188334346


In [52]:
categorical_encoded_test

,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3423,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3424,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
